# Build image classifier using transfer learning - Fine-tuning MobileNet with Keras

In [1]:
import theano
import keras

from keras import backend as k
from keras.models import Sequential
from keras.layers import Layer
from keras.layers import Dense
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import  ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from keras.layers import Flatten
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from keras.models import Model
import itertools
import numpy as np
import pandas as pd
%matplotlib inline

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
Using TensorFlow backend.


In [2]:
train_path = "data_set/dogscats/train"

test_path = "data_set/dogscats/sample/train"

validation_path = "data_set/dogscats/valid"

In [3]:
train_batches = ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input).flow_from_directory(train_path,target_size=(224,224),classes=["cats","dogs"],batch_size=10)

test_batches = ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input).flow_from_directory(test_path,target_size=(224,224),classes=["cats","dogs"],batch_size=10,shuffle=False)

validation_batches = ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input).flow_from_directory(validation_path,target_size=(224,224),classes=["cats","dogs"],batch_size=10)

Found 23000 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


# Modify Model

In [4]:
mobile = keras.applications.mobilenet.MobileNet()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [5]:
mobile.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
__________

In [6]:
x = mobile.layers[-6].output
prediction = Dense(2,activation='softmax')(x)
model = Model(input=mobile.input,output= prediction )

/home/v-nitin.gupta/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
__________

# Choosing Last 5 layer to train model 

In [8]:
for layer in model.layers[:-5]:
    layer.trainable = False


# Train Model 

In [9]:
model.compile(Adam(0.001),loss = categorical_crossentropy,metrics=["accuracy"])

In [10]:
model.fit_generator(train_batches,steps_per_epoch=4,validation_data=validation_batches,validation_steps= 2,epochs=30,verbose=2)

Instructions for updating:
Use tf.cast instead.
Epoch 1/30
 - 6s - loss: 0.4435 - acc: 0.7750 - val_loss: 1.7215 - val_acc: 0.6500
Epoch 2/30
 - 4s - loss: 0.7094 - acc: 0.8500 - val_loss: 0.4117 - val_acc: 0.8500
Epoch 3/30
 - 4s - loss: 0.2731 - acc: 0.8750 - val_loss: 1.3088 - val_acc: 0.7000
Epoch 4/30
 - 4s - loss: 0.2886 - acc: 0.9000 - val_loss: 3.6848 - val_acc: 0.6000
Epoch 5/30
 - 4s - loss: 0.4022 - acc: 0.8750 - val_loss: 0.8039 - val_acc: 0.8500
Epoch 6/30
 - 4s - loss: 0.3061 - acc: 0.9250 - val_loss: 1.5161 - val_acc: 0.7500
Epoch 7/30
 - 4s - loss: 1.1365 - acc: 0.8500 - val_loss: 1.0641 - val_acc: 0.8000
Epoch 8/30
 - 4s - loss: 0.3279 - acc: 0.9250 - val_loss: 0.6384 - val_acc: 0.9000
Epoch 9/30
 - 4s - loss: 0.2391 - acc: 0.9250 - val_loss: 0.0034 - val_acc: 1.0000
Epoch 10/30
 - 4s - loss: 0.1335 - acc: 0.9500 - val_loss: 0.2648 - val_acc: 0.9500
Epoch 11/30
 - 4s - loss: 0.5981 - acc: 0.7750 - val_loss: 0.2799 - val_acc: 0.9500
Epoch 12/30
 - 4s - loss: 0.0654 - ac

# Predict Cats and Dogs

In [11]:
test_lables = test_batches.classes

In [12]:
test_lables

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [13]:
test_batches.class_indices

{'cats': 0, 'dogs': 1}

In [14]:
prediction = model.predict_generator(test_batches,steps=1,verbose=0)